# Electric Forecasting Summary Results and Comparisons

## 01 Read in libraries and data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import datetime

In [2]:
# READ IN DATA FILES

# A. Predicted v actual files
path = "../Data/Modeling_results_predicted_v_actuals_reshaped.csv"
results = pd.read_csv(path)
results['datetime'] = pd.to_datetime(results.datetime)  # Convert to datetime object
results.datetime.asfreq("H")  #  informs data is hourly

# B. File with top 20 days in US
path = "../Data/Figures_top_20_days_us.csv"
usapeaks = pd.read_csv(path)
usapeaks['datetime'] = pd.to_datetime(usapeaks.datetime)  # Convert to datetime object
usapeaks.datetime.asfreq("H")  #  informs data is hourly
usapeaks['date'] = usapeaks['datetime'].dt.date  # Just making sure its a datetime object
del usapeaks['Unnamed: 0']
usapeaks.head()

#C. System info (characteristics)
path = "../Data/operating_system_names_and_info.csv"
systeminfo = pd.read_csv(path)
systeminfo.drop(columns=["ID", "ADDRESS", "CITY", "STATE", "ZIP", 
                         "COUNTRY", "NAICS_DESC", "SOURCE", "WEBSITE", "TELEPHONE"]
                , inplace = True )
systeminfo.rename(columns={'abbrev': 'abbr'}, inplace = True)
results.head()

,abbr,datetime,mw,predicted_ElasticNet,predicted_LinearRegression,predicted_RandomForest
0,AEC,2016-01-02 00:00:00,499.0,528.1673,546.5490,578.5564
1,AEC,2016-01-02 01:00:00,477.0,507.4926,520.2881,536.9399
2,AEC,2016-01-02 02:00:00,464.0,497.7633,518.9383,534.4162
3,AEC,2016-01-02 03:00:00,471.0,498.1755,517.4922,532.1400
4,AEC,2016-01-02 04:00:00,472.0,498.9360,518.6707,552.7252


## 02 R2 calculation for each individual location and model

In [3]:
# For loops 
systems = systeminfo['abbr']
models = ['predicted_LinearRegression', 'predicted_ElasticNet', 'predicted_RandomForest']

# A. R2 - linear models 
r2lfit = [ ]
for s in systems: 
    newr2 = r2_score(results.loc[results['abbr'] == s, :].mw, results.loc[results['abbr'] == s,:]['predicted_LinearRegression'])
    r2lfit.append(newr2)    
    
# B. R2 - Elastic Net
r2enet = [ ]
for s in systems: 
    newr2 = r2_score(results.loc[results['abbr'] == s, :].mw, results.loc[results['abbr'] == s,:]['predicted_ElasticNet'])
    r2enet.append(newr2)    

# C. R2 - Random Forest Regression
r2rfr = [ ]
for s in systems: 
    newr2 = r2_score(results.loc[results['abbr'] == s, :].mw, results.loc[results['abbr'] == s,:]['predicted_RandomForest'])
    r2rfr.append(newr2)  
    
# D. Convert it to a dataframe
r2dict = { 
    'abbr' : systems, 
    'r2_LinearRegression': r2lfit,
    'r2_ElasticNet': r2enet,
    'r2_RandomForest': r2rfr
}

r2results = pd.DataFrame(r2dict)

In [4]:
# MERGE WITH SYSTEM INFO, WRITE TO CSV and VIEW RESULTS

# A. Merge to system ino
r2results = pd.merge(r2results, systeminfo, on = 'abbr')

# B. Write to CSV
r2results.to_csv("../Data/Results - R2 scores.csv")

# C. View results
r2results

,abbr,r2_ElasticNet,r2_LinearRegression,r2_RandomForest,NAME,PEAK_MONTH,AVAIL_CAP,TOTAL_CAP,PEAK_LOAD,MIN_LOAD,SHAPE__Are,lon,lat
0,AZPS,0.925905,0.931492,0.950024,ARIZONA PUBLIC SERVICE COMPANY,6.0,71475.0,90516.0,7275.0,2260.0,4.170000e+11,-111.569906,34.316049
1,AECI,0.716633,0.721824,0.797304,"ASSOCIATED ELECTRIC COOPERATIVE, INC.",12.0,61242.0,73752.0,4521.0,1143.0,4.360000e+11,-93.391195,37.987117
2,AVA,0.847111,0.855342,0.890555,AVISTA CORPORATION,12.0,27943.0,28843.0,2273.0,961.0,2.490000e+11,-116.905492,47.039167
3,BANC,0.938120,0.940988,0.954852,BALANCING AUTHORITY OF NORTHERN CALIFORNIA,7.0,43244.0,53053.0,4426.0,2183.0,5.469669e+10,-121.899037,39.657983
4,BPAT,0.760271,0.765327,0.818188,BONNEVILLE POWER ADMINISTRATION,12.0,254766.0,478749.0,9739.0,4105.0,1.420000e+12,-117.424931,44.868673
5,CISO,0.920898,0.925068,0.941650,CALIFORNIA INDEPENDENT SYSTEM OPERATOR,7.0,659412.0,799412.0,46008.0,18354.0,6.740000e+11,-119.583372,37.359338
6,TPWR,0.784096,0.803845,0.857548,"CITY OF TACOMA, DEPARTMENT OF PUBLIC UTILITIES...",12.0,9022.0,15703.0,912.0,351.0,4.087245e+10,-122.258394,47.061846
7,DUK,0.742740,0.749123,0.806207,DUKE ENERGY CAROLINAS,7.0,261865.0,297312.0,20671.0,7067.0,1.400000e+11,-81.283339,35.350496
8,EPE,0.900564,0.903113,0.923419,EL PASO ELECTRIC COMPANY,7.0,16538.0,24719.0,1877.0,525.0,2.530000e+11,-103.703303,30.934747
9,ERCO,0.881476,0.883757,0.911323,"ELECTRIC RELIABILITY COUNCIL OF TEXAS, INC.",8.0,984731.0,1099555.0,71109.0,25051.0,6.980000e+11,-98.758981,31.220444


## 03 Top 20 days - Error and Comparisons

In [10]:
# A. Merge predictions to Top 20 days info
top20 = pd.merge(results, usapeaks, on = ['abbr', 'datetime'], how = 'inner')
del top20['mw_y']
top20.rename(columns={'mw_x': "Actual (MW)"}, inplace=True)

top20['% Error Linear Regression'] = (top20['predicted_LinearRegression'] / top20["Actual (MW)"] - 1) *100
top20['% Error Elastic Net'] = (top20['predicted_ElasticNet'] / top20["Actual (MW)"] - 1) *100
top20['% Error Random Forest'] = (top20['predicted_RandomForest'] / top20["Actual (MW)"] - 1) *100

In [11]:
# B. Summarize
top20.describe()

,Actual (MW),predicted_ElasticNet,predicted_LinearRegression,predicted_RandomForest,rank,hour,pctofmax,tempf,ID,PEAK_MONTH,AVAIL_CAP,TOTAL_CAP,PEAK_LOAD,MIN_LOAD,SHAPE__Are,lat,lon,% Error Linear Regression,% Error Elastic Net,% Error Random Forest
count,20174.000000,20174.000000,20174.000000,20174.000000,20174.000000,20174.000000,20174.000000,20174.000000,20174.000000,19955.000000,1.995500e+04,1.995500e+04,19955.000000,19526.000000,2.017400e+04,20174.000000,20174.000000,20174.000000,20174.000000,20174.000000
mean,11619.460395,11419.904268,11417.884540,11332.592522,10.586497,11.602211,63.793318,75.721435,14614.313076,7.637635,2.089762e+05,2.484346e+05,15790.023453,5747.570112,4.464846e+11,38.919957,-102.058687,-0.735692,-0.633212,-1.021358
std,24199.832661,23785.940936,23779.283461,23500.431174,5.819424,6.923289,60.363011,10.662186,11216.619448,3.138150,3.808414e+05,4.566458e+05,30221.789299,12031.544342,6.051092e+11,5.885068,16.636661,6.542621,6.782959,9.255928
min,-846.000000,-846.037400,-846.000000,-586.460000,1.000000,0.000000,17.808219,36.230000,189.000000,1.000000,1.032000e+03,3.693000e+03,98.000000,0.000000,2.462087e+09,27.634990,-122.755347,-45.974880,-45.121200,-35.903315
25%,942.000000,951.754175,949.059025,947.822300,6.000000,6.000000,51.074541,68.180000,6909.000000,7.000000,2.184800e+04,3.064000e+04,1808.000000,531.000000,5.469669e+10,34.316049,-117.638733,-4.576924,-4.560554,-4.144960
50%,2727.000000,2705.211500,2695.279000,2690.847000,11.000000,12.000000,61.405295,76.595000,14379.000000,7.000000,4.895200e+04,6.008600e+04,4101.000000,1355.000000,2.390000e+11,39.080949,-105.801990,-1.137404,-1.133651,-1.489713
75%,7016.750000,6975.546250,6979.757500,6917.062000,16.000000,18.000000,73.949718,83.840000,17716.000000,8.000000,1.605620e+05,1.847040e+05,9739.000000,4105.000000,5.870000e+11,43.832167,-84.572841,2.457944,2.617166,1.406958
max,152890.000000,158922.400000,158952.700000,147400.300000,20.000000,23.000000,3678.260870,99.230000,59504.000000,12.000000,1.825685e+06,2.113887e+06,152170.000000,57750.000000,2.720000e+12,48.341413,-71.403558,144.684231,150.499000,908.818600


In [12]:
# C. Write to CSV
top20.to_csv("../Data/Results Top 20 Days pcterror.csv")

In [13]:
top20.head()

,abbr,datetime,Actual (MW),predicted_ElasticNet,predicted_LinearRegression,predicted_RandomForest,rank,date,hour,pctofmax,...,AVAIL_CAP,TOTAL_CAP,PEAK_LOAD,MIN_LOAD,SHAPE__Are,lat,lon,% Error Linear Regression,% Error Elastic Net,% Error Random Forest
0,AEC,2016-07-22 00:00:00,476.0,496.3049,496.7977,492.1135,16.0,2016-07-22,0,40.683761,...,21848.0,30640.0,1916.0,531.0,1.370000e+11,31.863467,-86.389414,4.369265,4.265735,3.385189
1,AEC,2016-07-22 01:00:00,447.0,469.7742,471.4412,459.5305,16.0,2016-07-22,1,38.205128,...,21848.0,30640.0,1916.0,531.0,1.370000e+11,31.863467,-86.389414,5.467830,5.094899,2.803244
2,AEC,2016-07-22 02:00:00,431.0,446.0657,444.2859,443.3629,16.0,2016-07-22,2,36.837607,...,21848.0,30640.0,1916.0,531.0,1.370000e+11,31.863467,-86.389414,3.082575,3.495522,2.868422
3,AEC,2016-07-22 03:00:00,424.0,432.3484,430.9458,435.8154,16.0,2016-07-22,3,36.239316,...,21848.0,30640.0,1916.0,531.0,1.370000e+11,31.863467,-86.389414,1.638160,1.968962,2.786651
4,AEC,2016-07-22 04:00:00,420.0,432.8621,431.2835,442.7353,16.0,2016-07-22,4,35.897436,...,21848.0,30640.0,1916.0,531.0,1.370000e+11,31.863467,-86.389414,2.686548,3.062405,5.413167
